## Introduction to Flux Balance Analysis (FBA)

### Dr Sandy Macdonald, Bioinformatician in the TF
### s.macdonald@york.ac.uk

### Wednesday, 4th May, 2022

## Background

FBA, or Flux Balance Analysis is a metabolic modelling method widely used to build genome scale metabolic models.

Its first significant use was in a paper from David Fell in 1986 (https://doi.org/10.1042/bj2380781), where it was used to model fat synthesis in adipose tissue.

The technique was championed by Bernard Palsson, who has published many genome scale metabolic models, and improvements and extensions of the method over the past two decades.

I've used it in my PhD, in my postdoc with Gavin Thomas, and more recently as part of Gavin's MORF project.

## Quick summary of FBA

A model comprises, broadly, of a set of reactions with associated genes and metabolites in one or more "compartments" (more detail on that later...)

The reactions are represented as a set of balanced equations.

The set of balanced equations are solved using linear programming.

## Important terms

### Constraints

Each reaction has constraints applied, i.e. upper and lower bounds for the reaction's flux, which limits the solution space which has to be solved.

Often FBA is referred to as constraint-based modelling.

These constraints define the reversibility of the reaction.

### Objective function

An objective function is defined and optimised, given the set of equations and their constraints.

Usually this objective function is to maximise biomass production, but it could also be another function of the network, or it could be minimised. Multiple objective functions are possible.

The biomass reaction stoichiometry should be measured experimentally!

## Important terms

### Compartments

A model can have one or more compartments, between which metabolites can move via so-called exchange reactions (transport). Metabolites belong to one compartment, so exchange reactions between compartments have separate metabolites for each compartment on each side of the exchange reaction. 

These compartments are analagous to compartments within cells, e.g. extracellular space, periplasm, cytoplasm, etc.

### Uptakes and effluxes (AKA exchange reactions)

Exchange reactions that allow metabolites to enter and leave the model are called uptake and efflux reactions respectively.

An example of an uptake reaction would be glucose entering a model of _E. coli_ as a carbon source, and an example of an efflux reaction could be acetate leaving the model as a by-product.

These are one-sided reactions.

## Assumptions

Assumptions are an important part of any model!

One important assumption is that any model is incomplete, because you don't have perfect knowledge of its genes, proteins, and metabolic reactions. The extent of this incompleteness will vary.

By its nature, FBA assumes that the system is at steady state, i.e. that overall production equals consumption, although there are some hacky ways round this.

When combining FBA models with experimental data, experimental measurements are usually taken during log phase when growth rate is roughly linear or at pseudo steady state.

FBA takes no account of differing reaction kinetics.

The units of flux in FBA models are mmol/gram dry weight/hour.

## Summary from Nature FBA Primer

<div>
<img src="images/nature-fba-primer-image.jpeg" width="410"/>
</div>
(from https://doi.org/10.1038/nbt.1614)

## Where to find models

If you're lucky, then someone else will have built one for your organism already!

This is most likely if your organism is well-studied, like _E. coli_, _S. aureus_, _B. subtilis_, etc.

The BiGG Models database (http://bigg.ucsd.edu/models) is probably the best source of models. They use consistent notation for metabolites and reactions and make sure that the models are sensible.

Otherwise, you're going to have to build your own! More on that shortly...

## Model formats

All FBA models have the same main structure: lists of reactions, genes, metabolites, and compartments, along with some metadata about the model (ID, version).

There are two widely-used formats: SBML (Systems Biology Markup Language) and JSON (JavaScript Object Notation).

SBML is a flavour of XML specifically for metabolic models. It was the first format used for FBA models. Because of all the tags and chevrons, it's not the easiest to read by eye.

JSON is used for passing around bits of data in websites and web apps, but its structure lends itself well to FBA models also, and it's now used routinely for this purpose. We'll focus on JSON here.

## Example reaction - JSON

Here's an example of a reaction from a JSON model, the _E. coli_ core metabolism model from the BiGG database (http://bigg.ucsd.edu/models/e_coli_core):

```json
{
"id":"FBA",
"name":"fructose-bisphosphate aldolase",
"metabolites":{
"dhap_c":1.0,
"fdp_c":-1.0,
"g3p_c":1.0
},
"lower_bound":-1000.0,
"upper_bound":1000.0,
"gene_reaction_rule":"b1773 or b2097 or b2925",
"annotation":{
"bigg.reaction":"FBA"
}
},
```

Left hand side reactants have negative stoichiometries, right hand side products have positive ones.

Gene reaction rules can be `and` or `or` or some combination of both.

## Example reaction - SBML

```xml
<reaction metaid="meta_R_FBA" id="R_FBA" name="fructose-bisphosphate aldolase" reversible="true" fast="false" fbc:lowerFluxBound="cobra_default_lb" fbc:upperFluxBound="cobra_default_ub">
  <annotation>
    <rdf:RDF xmlns:rdf="http://www.w3.org/1999/02/22-rdf-syntax-ns#" xmlns:dcterms="http://purl.org/dc/terms/" xmlns:vCard="http://www.w3.org/2001/vcard-rdf/3.0#" xmlns:vCard4="http://www.w3.org/2006/vcard/ns#" xmlns:bqbiol="http://biomodels.net/biology-qualifiers/" xmlns:bqmodel="http://biomodels.net/model-qualifiers/">
      <rdf:Description rdf:about="#meta_R_FBA">
        <bqbiol:is>
          <rdf:Bag>
            <rdf:li rdf:resource="https://identifiers.org/bigg.reaction/FBA"/>
          </rdf:Bag>
        </bqbiol:is>
      </rdf:Description>
    </rdf:RDF>
  </annotation>
  <listOfReactants>
    <speciesReference species="M_fdp_c" stoichiometry="1" constant="true"/>
  </listOfReactants>
  <listOfProducts>
    <speciesReference species="M_dhap_c" stoichiometry="1" constant="true"/>
    <speciesReference species="M_g3p_c" stoichiometry="1" constant="true"/>
  </listOfProducts>
  <fbc:geneProductAssociation>
    <fbc:or>
      <fbc:geneProductRef fbc:geneProduct="G_b1773"/>
      <fbc:geneProductRef fbc:geneProduct="G_b2097"/>
      <fbc:geneProductRef fbc:geneProduct="G_b2925"/>
    </fbc:or>
  </fbc:geneProductAssociation>
</reaction>
```

## One method of building an FBA model

My preferred process is:

1. Start with the most closely-related organism for which an FBA model exists.
2. Use homology to match as many metabolic genes between the organisms (BLAST).
3. Transfer those reactions which have gene matches to your new model.
4. Attempt to fill gaps.

This, of course, relies upon your organism having a well-annotated genome to work from.

Ideally, you'd also have gene expression or proteomic data to identify missing enzymes/reactions. Even better if you're able to express and characterise the enzyme activities experimentally.

Many FBA models have not insignificant numbers of gap-filled reactions (without gene evidence).

## FBA software

My tool of choice is the COBRA toolbox, which has a Python library called cobrapy (https://opencobra.github.io/cobrapy/).

It stems from the Palsson lab's original COBRA toolbox which ran in MATLAB (I used this in my postdoc with Gavin in 2008!)

It has a whole bunch of functionality:

* Reading and writing FBA models in SBML, JSON, YAML, MATLAB, and Python's pickle format
* Building models from scratch, i.e. adding reactions, metabolites, genes, setting the objective function
* Solving models (running simulations)
* Simulating gene deletions (essentiality)
* Predicting minimal media
* Flux variability analysis (FVA)

The cobrapy documentation (https://cobrapy.readthedocs.io/en/latest/) is excellent, so do read it if you're interested.

## cobrapy examples

### Building a simple model

Let's look at how to create an empty model, and add a reaction and metabolites:

In [2]:
from cobra import Model, Reaction, Metabolite


model = Model('toy_model')  # Create a new model

r1 = Reaction('R1')  # Create a new reaction
r1.name = 'Reaction 1'  # Name it
r1.lower_bound = -1000.0  # Set the lower and
r1.upper_bound = 1000.0  # upper bounds

# Create three metabolites
a = Metabolite(
    'A_c',
    formula='',
    name='metabolite A',
    compartment='c')

b = Metabolite(
    'B_c',
    formula='',
    name='metabolite B',
    compartment='c')

c = Metabolite(
    'C_c',
    formula='',
    name='metabolite C',
    compartment='c')

# Add the three metabolites to the reaction,
# with their stoichiometries, -ve for reactants,
# +ve for products
r1.add_metabolites({
    a: -1.0,
    b: 2.0,
    c: 0.5
})

# Add the reaction to the model
model.add_reactions([r1])

In [6]:
# Printing a reaction gives a neat summary
print(r1)

# model.reactions is a list of all the reaction instances
print(f"Reactions: {[r.id for r in model.reactions]}")

# model.metabolites is a list of all the metabolite instances
print(f"Metabolites: {[m.id for m in model.metabolites]}")

# If we change r1 to be irreversible and then print it again...
r1.lower_bound = 0

# ... then the printed summary changes accordingly!
print(r1)

R1: A_c <=> 2.0 B_c + 0.5 C_c
Reactions: ['R1']
Metabolites: ['A_c', 'B_c', 'C_c']
R1: A_c --> 2.0 B_c + 0.5 C_c


### Reading in models

We can read FBA models in to cobrapy and then work with them, run simulations, etc.

To read a JSON format model in, we use the `load_json_model()` function from the `cobra.io` module.

To read in an SBML format model, we use the `read_sbml_model()` function from the `cobra.io` module (I  have no idea why one is `load` and the other is `read`...

Let's read in the _E. coli_ core metabolism model now and try some things.

In [32]:
# import cobra.io

# Read in the E. coli core model in JSON format
model = cobra.io.load_json_model('e_coli_core.json')

# Print a useful summary of the model. This also
# solves the model.
print(model.summary())

Objective
1.0 Biomass_Ecoli_core = 0.8739215069684307

Uptake
------
Metabolite    Reaction  Flux  C-Number  C-Flux
  glc__D_e EX_glc__D_e    10         6 100.00%
     nh4_e    EX_nh4_e 4.765         0   0.00%
      o2_e     EX_o2_e  21.8         0   0.00%
      pi_e     EX_pi_e 3.215         0   0.00%

Secretion
---------
Metabolite Reaction   Flux  C-Number  C-Flux
     co2_e EX_co2_e -22.81         1 100.00%
     h2o_e EX_h2o_e -29.18         0   0.00%
       h_e   EX_h_e -17.53         0   0.00%



### Affecting biomass flux

In the case of this simple model, the carbon source (glucose/fructose) has a direct effect on the biomass flux (growth rate) of the model.

The glucose uptake (to the model) is an exchange reaction which we looked at earlier, a one-sided reaction.

Let's look at how we can manipulate that and see the resulting effect on biomass flux.

In [38]:
# Find the lower bound of the glucose uptake reaction.
# This is the growth-limiting step.
print(model.reactions.get_by_id("EX_glc__D_e").lower_bound)

# Let's solve the model to find the biomass flux.
print(model.optimize())

# Double the amount of glucose uptake
model.reactions.get_by_id("EX_glc__D_e").lower_bound = -20

print()

# Solve the model again
print(model.optimize())

-10.0
<Solution 0.874 at 0x7fe280d9b9d0>

<Solution 1.791 at 0x7fe280d9bac0>


### Gene deletions (single)

Let's do something a bit more interesting, and run a single gene deletion analysis.

This goes through each gene in turn and knocks it out, and then solves the model to find the biomass flux.

The `single_gene_deletion()` function returns a pandas dataframe with the gene knocked out, growth rate (biomass flux), and whether the solution is optimal (non-zero) or infeasible.

Genes which result in an infeasible solution when knocked out are essential.

In [39]:
from cobra.flux_analysis import single_gene_deletion

# Run the single gene deletion and store the results
deletion_results = single_gene_deletion(model)

# Sort the results table by the status column to find the
# infeasible knockouts.
print(deletion_results.sort_values("status", ascending=True)[:10])

         ids    growth      status
127  {b2415}       NaN  infeasible
22   {b2416}       NaN  infeasible
88   {b0356}  1.790569     optimal
89   {b2286}  0.516254     optimal
90   {b2287}  0.516254     optimal
91   {b3956}  1.784060     optimal
92   {b3603}  1.790569     optimal
87   {b3870}  1.790569     optimal
0    {b2280}  0.516254     optimal
94   {b2458}  1.790569     optimal


In [40]:
# Let's have a look at what reactions those two essential
# genes belong to:
essential_genes = ["b2415", "b2416"]

# Loop through all the reactions and check whether their 
# genes have any overlap with the essential genes, printing
# their name and id if they do
for r in model.reactions:
    genes = r.genes
    gene_ids = [g.id for g in genes]
    if len(set(essential_genes).intersection(gene_ids)):
        print(r.id, r.name)

FRUpts2 R Fructose transport via PEPPyr PTS-f6p - generating
GLCpts D-glucose transport via PEP:Pyr PTS


### Flux Variability Analysis (FVA)

Lastly, we'll look at flux variability analysis.

This method computes the range of possible flux that each reaction can have while maintaining the optimal biomass flux. It's useful for identifying redundant reactions and, conversely, essential reactions.

Reactions with a small amount of flux variability are likely to be more important, whereas those with a larger range are less likely to be important, as changing their flux has no effect on optimal biomass flux.

Let's look at an example:

In [45]:
import cobra.flux_analysis

# Run the FVA and store the results
fva = cobra.flux_analysis.variability.flux_variability_analysis(model)

# Create a new column that is the difference, i.e. the
# amount of variability possible.
fva["difference"] = abs(fva["maximum"] - fva["minimum"])

# Sort by that new column
fva.sort_values("difference", ascending=False)

,minimum,maximum,difference
FRD7,0.000000,990.797010,9.907970e+02
SUCDi,9.202990,1000.000000,9.907970e+02
AKGDH,9.202990,9.202990,5.575984e-12
ICDHyr,11.134835,11.134835,5.446310e-12
PPC,5.131054,5.131054,5.114131e-12
...,...,...,...
EX_pi_e,-6.586966,-6.586966,1.776357e-14
PIt2r,6.586966,6.586966,1.509903e-14
Biomass_Ecoli_core,1.790569,1.790569,5.995204e-15
MALt2_2,0.000000,0.000000,0.000000e+00


In [47]:
# Let's look at what the two reactions with the 
# largest amount of flux variability are...

frd7 = model.reactions.get_by_id("FRD7")
print(frd7)

sucdi = model.reactions.get_by_id("SUCDi")
print(sucdi)

FRD7: fum_c + q8h2_c --> q8_c + succ_c
SUCDi: q8_c + succ_c --> fum_c + q8h2_c


## More extensions to FBA

### Production envelopes/phenotypic phase planes

This allows the interplay between two metabolites (usually a substrate and a product) and the objective flux to be investigated, e.g. what is the effect of both glucose and oxygen availability on production of acetate (example in the cobrapy docs)?

### Multiple gene knockouts

cobrapy has methods for double gene deletions and you could build your own mutiple gene deletion methods (although the permutations explode quickly!)

### Dynamic FBA (dFBA)

Dynamic FBA allows dynamic changes in external metabolite concentrations to be incorporated into an FBA model. _In silico_ equivalent of time course metabolomic data. Allows, for example, the glucose consumption to be plotted against growth rate through time (example in the cobrapy docs).

## Real world examples

In 2009, we published (https://doi.org/10.1186/1752-0509-3-24) an FBA model of _Buchnera aphidicola_, the obligate bacterial symbiont of the pea aphid.

The metabolic network (and indeed genome) of _Buchnera_ is a subset of _E. coli_, which made building the model much more straighforward than usual!

We were able to show: 

* the fragility of the network (~95% of its genes are essential, compared to ~20% for _E. coli_)
* that essential amino acid synthesis is coupled to growth
* that the essential amino acid profile can be altered by altering supply of carbon and nitrogen sources

### Buchnera gene essentiality (gene deletion analysis)

<div>
<img src="images/buchnera-gene-essentiality.jpeg" width="300"/>
</div>

A: _E. coli_, B: _Buchnera_

## Summing up

FBA is a useful metabolic modelling method that can be used to understand an organism's metabolism in detail.

It avoids the headaches of reaction kinetics, but you should be careful to know what assumptions your model makes and to consider the quality of the data used to build the model.

Many pre-built FBA models exist already, especially if your favourite organism is well-studied.

You can build your own genome-scale metabolic model by homology.

The cobrapy Python library is the go-to method for working with FBA models and provides many ways to interrogate your metabolic model .

## Useful resources

Here are some useful resources to learn more about what I've spoken about today:

* Nature Primer on Flux Balance Analysis: [https://doi.org/10.1038/nbt.1614](https://doi.org/10.1038/nbt.1614)
* BiGG Models database: [http://bigg.ucsd.edu/models](http://bigg.ucsd.edu/models)
* cobrapy documentation: [https://cobrapy.readthedocs.io/en/latest/](https://cobrapy.readthedocs.io/en/latest/)


## Contact details

Please don't hesitate to get in touch if you need any help or advice: s.macdonald@york.ac.uk